In [ ]:
import numpy as np
import pandas as pd
from keras.applications.xception import Xception
from keras.preprocessing.image import load_img, img_to_array
#from keras.utils import load_img,img_to_array
from keras.layers import Dense, Input
from keras.models import Model
from keras.utils import to_categorical
from sklearn.metrics import classification_report

Load data

In [ ]:
train_data = pd.read_csv('task_damage_text_img_train.csv')
test_data = pd.read_csv('task_damage_text_img_test.csv')
val_data = pd.read_csv('task_damage_text_img_dev.csv')

Load image data

In [ ]:
img_width, img_height = 100, 100
num_classes = 3
img_train = []
img_test = []
img_val = []

In [ ]:
for filename in train_data['image']:
    img = load_img( filename, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_train.append(img_array)

In [ ]:
for filename in test_data['image']:
    img = load_img( filename, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_test.append(img_array)

In [ ]:
for filename in val_data['image']:
    img = load_img( filename, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_val.append(img_array)

In [ ]:
img_train = np.array(img_train)
img_test = np.array(img_test)
img_val = np.array(img_val)

Preprocess data

In [ ]:
img_train = img_train / 255.0
img_test = img_test / 255.0
img_val = img_val / 255.0

In [ ]:
tempTrain=[]
tempTest=[]
tempVal=[]

for i in range(0, 2468):
    if train_data['label'][i] == 'severe_damage':
        tempTrain.append(0)
    elif train_data['label'][i] == 'mild_damage':
        tempTrain.append(1)
    elif train_data['label'][i] == 'little_or_no_damage':
        tempTrain.append(2)

for i in range(0, 529):
    if test_data['label'][i] == 'severe_damage':
        tempTest.append(0)
    elif test_data['label'][i] == 'mild_damage':
        tempTest.append(1)
    elif test_data['label'][i] == 'little_or_no_damage':
        tempTest.append(2)

for i in range(0, 529):
    if val_data['label'][i] == 'severe_damage':
        tempVal.append(0)
    elif val_data['label'][i] == 'mild_damage':
        tempVal.append(1)
    elif val_data['label'][i] == 'little_or_no_damage':
        tempVal.append(2)
labels_train = tempTrain
labels_test = tempTest
labels_val = tempVal
Y_train = to_categorical(labels_train, num_classes=num_classes)
Y_test = to_categorical(labels_test, num_classes=num_classes)
Y_val = to_categorical(labels_val, num_classes=num_classes)

Define Xception model

In [ ]:
xception_input = Input(shape=(img_width, img_height, 3))
xception_model = Xception(weights='imagenet', include_top=False, input_tensor=xception_input, pooling='max')
xception_output = Dense(num_classes, activation='softmax')(xception_model.output)
xception_model = Model(inputs=xception_input, outputs=xception_output)

Compile the model

In [ ]:
xception_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Train the model

In [ ]:
xception_model.fit(img_train, Y_train, batch_size=40, epochs=10, validation_data=(img_val, Y_val))

Evaluate the model

In [ ]:
_, accuracy = xception_model.evaluate(img_test, Y_test)
print('Accuracy: %.2f' % (accuracy*100))

Make predictions on the test dataset

In [ ]:
y_pred = xception_model.predict(img_test)
y_pred_classes = np.argmax(y_pred, axis=1)

Convert labels from one-hot encoding to integers

In [ ]:
y_test_classes = np.argmax(Y_test, axis=1)

Generate classification report

In [ ]:
report = classification_report(y_test_classes, y_pred_classes)
print(report)